In [1]:
import requests 
from requests.auth import HTTPBasicAuth
import json
import numpy as np
import pandas as pd

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from joblib import parallel_backend

from time import time
from tensorflow.keras.callbacks import TensorBoard
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.linear_model import LinearRegression

In [4]:
import json

### get data

In [5]:
api_key = '1767f88c-6e71-4d1d-81e6-f6324c83b1e3'

In [8]:
def get_pred(pred):
  # Your api key
  server_url = 'https://esgf-server.herokuapp.com'
  # Service name (see https://esgf-server-staging.herokuapp.com/docs)
  service_name = 'get_inference_data'
  # Service parameters.
  params = f"?api_key={api_key}"

  # Request header.
  headers = {
        'Accept': 'application/json',
        'Authorization' : f"Bearer {api_key}"
  }

  response = requests.get(f"{server_url}/{service_name}{params}", headers=headers)

  # Logs.
  print(response.text)

  # Assert valid answer (see https://developer.mozilla.org/fr/docs/Web/HTTP/Status/200)
  #assert response.status_code == 200

  # Server answer as json. Dictionary with keys 'id' and 'x'
  json_answer = response.json()

  # Example id
  example_id = json_answer['id']

  # Parse str answer as json object
  json_answer = json.loads(json_answer['x'])

  # Frame answer
  pd_sample = pd.DataFrame(json_answer)
    
  liste = list(list(pred[-1]))
  preds = ",".join(str(x) for x in liste)
  print("Sample ID='{}' has shape {}".format(example_id, pd_sample.shape))
  return example_id, preds, pd_sample

## get train data

In [6]:
df = pd.read_csv('TRAIN.csv')

In [7]:
df.head()

,wconfid,pctid,x,y,z
0,1,20,1.004,0.090,-0.125
1,1,20,1.004,-0.043,-0.125
2,1,20,0.969,0.090,-0.121
3,1,20,0.973,-0.012,-0.137
4,1,20,1.000,-0.016,-0.121


In [9]:
X = df.iloc[:,:1]

In [10]:
Y = df.iloc[:,-3:]

### Split dataset

In [11]:
X = np.array(X)
Y = np.array(Y)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, Y) 

In [13]:
x_test

array([[2],
       [2],
       [2],
       ...,
       [1],
       [1],
       [3]], dtype=int64)

## Model

In [14]:
model = LinearRegression()

In [15]:
model.fit(x_train, y_train)

LinearRegression()

In [16]:
Y_pred = model.predict(x_test)

In [17]:
Y_pred

array([[ 1.00047036,  0.00633179, -0.11849573],
       [ 1.00047036,  0.00633179, -0.11849573],
       [ 1.00047036,  0.00633179, -0.11849573],
       ...,
       [ 1.0090157 ,  0.00128261, -0.12032948],
       [ 1.0090157 ,  0.00128261, -0.12032948],
       [ 0.99192502,  0.01138097, -0.11666199]])

In [18]:
y_test

array([[ 1.211, -0.035, -0.223],
       [ 0.195,  1.203, -0.223],
       [ 0.68 , -0.312, -1.012],
       ...,
       [ 0.984,  0.859, -0.941],
       [ 3.008, -2.371,  0.187],
       [ 1.   ,  0.105, -0.352]])

In [19]:
id, preds, pd_sample = get_pred(Y_pred)

{"id":"REdwdWjoM5o8deW9","x":"{\"wconfid\":{\"0\":3,\"1\":3,\"2\":3,\"3\":3,\"4\":3,\"5\":3,\"6\":3,\"7\":3,\"8\":3,\"9\":3,\"10\":3,\"11\":3,\"12\":3,\"13\":3,\"14\":3,\"15\":3,\"16\":3,\"17\":3,\"18\":3,\"19\":3,\"20\":3,\"21\":3,\"22\":3,\"23\":3,\"24\":3,\"25\":3,\"26\":3,\"27\":3,\"28\":3,\"29\":3,\"30\":3,\"31\":3},\"pctid\":{\"0\":100,\"1\":100,\"2\":100,\"3\":100,\"4\":100,\"5\":100,\"6\":100,\"7\":100,\"8\":100,\"9\":100,\"10\":100,\"11\":100,\"12\":100,\"13\":100,\"14\":100,\"15\":100,\"16\":100,\"17\":100,\"18\":100,\"19\":100,\"20\":100,\"21\":100,\"22\":100,\"23\":100,\"24\":100,\"25\":100,\"26\":100,\"27\":100,\"28\":100,\"29\":100,\"30\":100,\"31\":100},\"x\":{\"0\":0.941,\"1\":0.98,\"2\":0.926,\"3\":0.992,\"4\":0.688,\"5\":1.027,\"6\":1.027,\"7\":1.273,\"8\":0.961,\"9\":0.816,\"10\":1.004,\"11\":1.012,\"12\":1.156,\"13\":0.891,\"14\":1.121,\"15\":0.832,\"16\":1.051,\"17\":0.688,\"18\":1.086,\"19\":0.965,\"20\":0.871,\"21\":1.07,\"22\":1.102,\"23\":0.98,\"24\":0.867,\"25

In [20]:
preds

'0.9919250239746649,0.011380972641668995,-0.1166619894224334'

In [22]:
for i in range(300):
    id, preds, pd_sample = get_pred(Y_pred)
    server_url = 'https://esgf-server.herokuapp.com/' #Prod
    #server_url = 'https://esgf-server-staging.herokuapp.com'
    # Service name (see https://esgf-server-staging.herokuapp.com/docs)
    service_name = 'predict'
    params = f"?api_key={api_key}&example_id={id}&prediction={preds}"
    headers = {
          'Accept': 'application/json',
          'Authorization' : f"Bearer {api_key}"
    }
    response = requests.get(f"{server_url}/{service_name}{params}", headers=headers)
    print(f"{server_url}/{service_name}{params}")
    print(response.text)
    #assert response.status_code == 200

{"id":"aK3zHI0CJEAkZZXZ","x":"{\"wconfid\":{\"0\":1,\"1\":1,\"2\":1,\"3\":1,\"4\":1,\"5\":1,\"6\":1,\"7\":1,\"8\":1,\"9\":1,\"10\":1,\"11\":1,\"12\":1,\"13\":1,\"14\":1,\"15\":1,\"16\":1,\"17\":1,\"18\":1,\"19\":1,\"20\":1,\"21\":1,\"22\":1,\"23\":1,\"24\":1,\"25\":1,\"26\":1,\"27\":1,\"28\":1,\"29\":1,\"30\":1,\"31\":1},\"pctid\":{\"0\":80,\"1\":80,\"2\":80,\"3\":80,\"4\":80,\"5\":80,\"6\":80,\"7\":80,\"8\":80,\"9\":80,\"10\":80,\"11\":80,\"12\":80,\"13\":80,\"14\":80,\"15\":80,\"16\":80,\"17\":80,\"18\":80,\"19\":80,\"20\":80,\"21\":80,\"22\":80,\"23\":80,\"24\":80,\"25\":80,\"26\":80,\"27\":80,\"28\":80,\"29\":80,\"30\":80,\"31\":80},\"x\":{\"0\":1.152,\"1\":0.824,\"2\":1.645,\"3\":1.859,\"4\":1.059,\"5\":2.227,\"6\":1.859,\"7\":0.801,\"8\":0.215,\"9\":1.359,\"10\":0.758,\"11\":1.086,\"12\":0.078,\"13\":-1.125,\"14\":0.801,\"15\":0.871,\"16\":0.695,\"17\":-0.297,\"18\":1.848,\"19\":0.914,\"20\":-0.281,\"21\":0.449,\"22\":0.832,\"23\":1.117,\"24\":0.121,\"25\":-0.355,\"26\":1.395,\"2

AssertionError: 